# init 
importing libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

# Reading_cvs

In [2]:
df = pd.read_csv('../data/athletes_sport_details.csv')
country_df = pd.read_csv('../data/country_ranks.csv')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,4,False
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,18,False
2,3,Gunnar Nielsen Aaby,M,24.0,191.0,85.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,12,False
3,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,20,False
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,20,False


# Making a model just for Gymnastics athletics

In [3]:
Gymnastics_df = df[df['Sport'] == 'Gymnastics']
Gymnastics_df.head() , Gymnastics_df.describe()

(    ID                     Name Sex   Age  Height  Weight     Team  NOC  \
 32  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 33  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 34  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 35  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 36  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 
           Games  Year  Season    City       Sport  \
 32  1948 Summer  1948  Summer  London  Gymnastics   
 33  1948 Summer  1948  Summer  London  Gymnastics   
 34  1948 Summer  1948  Summer  London  Gymnastics   
 35  1948 Summer  1948  Summer  London  Gymnastics   
 36  1948 Summer  1948  Summer  London  Gymnastics   
 
                                      Event   Medal  Sport_Rank  \
 32  Gymnastics Men's Individual All-Around  Bronze          14   
 33        Gymnastics Men's Team All-Around    Gold          14   
 34         Gym

we need a little sample so we use last 25% of 18k => 4K

In [4]:
Gymnastics_df_5000_sample = Gymnastics_df[Gymnastics_df['Year'] >= 2000]
Gymnastics_df_5000_sample["ID"] = range(1, len(Gymnastics_df_5000_sample) + 1)
Gymnastics_df_5000_sample.to_csv('../data/Gymnastics_sample.csv')
Gymnastics_df_5000_sample

C:\Users\aamyr\AppData\Local\Temp\ipykernel_13956\2422990920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gymnastics_df_5000_sample["ID"] = range(1, len(Gymnastics_df_5000_sample) + 1)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal
107,1,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False
108,2,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,NaN,14,False
109,3,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,NaN,14,False
110,4,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,14,False
111,5,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Rings,NaN,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205656,4868,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN,14,True
205657,4869,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN,14,True
205658,4870,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,14,True
205659,4871,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Rings,NaN,14,True


In [5]:
# Dictionary to store the last cumulative count per name
last_count = {}

# Initialize the column
Gymnastics_df_5000_sample['Count'] = 0

# Sort by Name and Year to ensure proper ordering
Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.sort_values(['Name', 'Year']).reset_index(drop=True)

# Iterate through rows
for name, group in Gymnastics_df_5000_sample.groupby('Name', sort=False):
    # Keep track of cumulative count for this name
    count = 0
    for year, year_group in group.groupby('Year', sort=False):
        # Assign the current count to all rows of this year
        indices = year_group.index
        Gymnastics_df_5000_sample.loc[indices, 'Count'] = count
        # Update cumulative count for next year
        count += len(year_group)

Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.sort_values(by='ID').reset_index(drop=True)
Gymnastics_df_5000_sample

C:\Users\aamyr\AppData\Local\Temp\ipykernel_13956\2131119701.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gymnastics_df_5000_sample['Count'] = 0


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal,Count
0,1,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False,0
1,2,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,NaN,14,False,0
2,3,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,NaN,14,False,0
3,4,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,14,False,0
4,5,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Rings,NaN,14,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4867,4868,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN,14,True,8
4868,4869,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN,14,True,8
4869,4870,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,14,True,8
4870,4871,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Rings,NaN,14,True,8


making ranking for every country based on economy and technology

In [6]:
x=Gymnastics_df_5000_sample["Team"].unique()
len(x)

75

In [7]:
result = pd.merge(Gymnastics_df_5000_sample, country_df, how="left", on=["Team"])
result = result.drop(columns=["Sport_Rank","Year"])
Gymnastics_df_5000_sample = result[["ID","Name","Sex","Age","Height","Weight","Team","Development_Status","Rank","NOC","Games","Season","City","Sport","Event","Has_Previous_Medal","Count","Medal"]]

In [8]:
Gymnastics_df_5000_sample.to_csv('../data/Gymnastics_sample.csv')
Gymnastics_df_5000_sample

,ID,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,1,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,Developed,1,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,False,0,NaN
1,2,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,Developed,1,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,False,0,NaN
2,3,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,Developed,1,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,False,0,NaN
3,4,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,Developed,1,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,False,0,NaN
4,5,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,Developed,1,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Rings,False,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4867,4868,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,Developing,3,UKR,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,True,8,NaN
4868,4869,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,Developing,3,UKR,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,True,8,NaN
4869,4870,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,Developing,3,UKR,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Horizontal Bar,True,8,NaN
4870,4871,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,Developing,3,UKR,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Rings,True,8,NaN


- we gather some data now lets shift to other processes
- endcoding

In [9]:
Gymnastics_df_5000_sample["Sex"] = Gymnastics_df_5000_sample["Sex"].map({"M": 0, "F": 1})
Gymnastics_df_5000_sample["Has_Previous_Medal"] = Gymnastics_df_5000_sample["Has_Previous_Medal"].map({False: 0, True: 1})
Gymnastics_df_5000_sample["Rank"] = Gymnastics_df_5000_sample["Development_Status"].map({
    "Developed": 3,
    "Upper-Middle": 2,
    "Developing": 1
})

In [10]:
Gymnastics_df_5000_sample.head()

,ID,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,1,Nstor Abad Sanjun,0,23.0,167.0,64.0,Spain,Developed,3,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,0,0,NaN
1,2,Nstor Abad Sanjun,0,23.0,167.0,64.0,Spain,Developed,3,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,0,0,NaN
2,3,Nstor Abad Sanjun,0,23.0,167.0,64.0,Spain,Developed,3,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,0,0,NaN
3,4,Nstor Abad Sanjun,0,23.0,167.0,64.0,Spain,Developed,3,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,0,0,NaN
4,5,Nstor Abad Sanjun,0,23.0,167.0,64.0,Spain,Developed,3,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Rings,0,0,NaN


- i use Z-score formula for standardizition (scaling)
- x′=(x−μ)/σ

In [13]:
num_cols = ["Age", "Height", "Weight","Count","Rank","Has_Previous_Medal"]

for col in num_cols:
    mean = Gymnastics_df_5000_sample[col].mean()
    std = Gymnastics_df_5000_sample[col].std()
    Gymnastics_df_5000_sample[col] = (Gymnastics_df_5000_sample[col] - mean) / std
Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.drop(columns=["ID"])
Gymnastics_df_5000_sample.describe()

,Sex,Age,Height,Weight,Rank,Has_Previous_Medal,Count
count,4872.000000,4.872000e+03,4.872000e+03,4.872000e+03,4.872000e+03,4.872000e+03,4.872000e+03
mean,0.437603,-1.750105e-17,5.833684e-18,-1.166737e-17,-1.166737e-17,-4.958632e-17,-1.166737e-17
std,0.496142,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,0.000000,-1.805789e+00,-3.262067e+00,-2.656875e+00,-2.353906e+00,-3.216979e-01,-4.850942e-01
25%,0.000000,-8.468479e-01,-6.043324e-01,-7.250924e-01,-8.598687e-01,-3.216979e-01,-4.850942e-01
50%,0.000000,-1.276423e-01,8.898960e-02,1.442096e-01,6.341685e-01,-3.216979e-01,-4.850942e-01
75%,1.000000,5.915634e-01,6.667579e-01,7.237443e-01,6.341685e-01,-3.216979e-01,-4.850942e-01
max,1.000000,4.667062e+00,2.631170e+00,3.041883e+00,6.341685e-01,3.107869e+00,6.179657e+00


In [14]:
Gymnastics_df_5000_sample.head()

,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,Nstor Abad Sanjun,0,0.351828,0.666758,0.820333,Spain,Developed,0.634169,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,-0.321698,-0.485094,NaN
1,Nstor Abad Sanjun,0,0.351828,0.666758,0.820333,Spain,Developed,0.634169,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,-0.321698,-0.485094,NaN
2,Nstor Abad Sanjun,0,0.351828,0.666758,0.820333,Spain,Developed,0.634169,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,-0.321698,-0.485094,NaN
3,Nstor Abad Sanjun,0,0.351828,0.666758,0.820333,Spain,Developed,0.634169,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,-0.321698,-0.485094,NaN
4,Nstor Abad Sanjun,0,0.351828,0.666758,0.820333,Spain,Developed,0.634169,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Rings,-0.321698,-0.485094,NaN
